# Mount Drive



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Importing Libraries and initializing stopwords and stemmer


In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf

import warnings
warnings.filterwarnings("ignore")

import nltk

import re
from bs4 import BeautifulSoup
from tqdm import tqdm
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score, confusion_matrix
from sklearn.metrics import classification_report


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

# Reading Data


In [ ]:
hasoc_data = pd.read_csv("drive/MyDrive/ICHCL_baseline/hasoc2021-train/Hasoc2021_train.csv") #Your respective address
hasoc_data.head()


,Unnamed: 0,_id,text,task_1,task_2
0,4986,60c5d6bf5659ea5e55defa2c,@wealth if you made it through this &amp;&amp;...,HOF,PRFN
1,3394,60c5d6bf5659ea5e55def461,Technically that's still turning back the cloc...,HOF,OFFN
2,1310,60c5d6bf5659ea5e55defaad,@VMBJP @BJP4Bengal @BJP4India @narendramodi @J...,NOT,NONE
3,3390,60c5d6bf5659ea5e55def419,@krtoprak_yigit Soldier of Japan Who has dick ...,HOF,OFFN
4,4626,60c5d6bf5659ea5e55def7fa,@blueheartedly You'd be better off asking who ...,HOF,OFFN


In [ ]:
aakash_test = pd.read_csv("drive/MyDrive/ICHCL_baseline/Test/aakash_300.csv")

In [ ]:
hasoc_data.describe()

,Unnamed: 0
count,3843.000000
mean,2558.799375
std,1477.088070
min,1.000000
25%,1276.500000
50%,2574.000000
75%,3828.000000
max,5121.000000


In [ ]:
tweets = hasoc_data.text
y_task1 = hasoc_data.task_1
y_task2 = hasoc_data.task_2

In [ ]:
test_hasoc = pd.read_csv("drive/MyDrive/ICHCL_baseline/Test/en_Hasoc2021_test_task1.csv")
test_hasoc.head()

,_id,text
0,60c5d6bf5659ea5e55deffcb,Fewer people coming in for vaccinations. So sa...
1,60c5d6bf5659ea5e55df028c,@MattHancock This may all be true. But... What...
2,60c5d6bf5659ea5e55def377,@Layla_EFC I’ve unfollowed him the wanker
3,60c5d6bf5659ea5e55def4c7,You guys are losing it all over the world. The...
4,60c5d6bf5659ea5e55df01a6,"And thus death laughs... It is sad merriment, ..."


# Basic Preprocessing of Tweets


In [ ]:
def decontract(text):
    text = re.sub(r"won\'t", "will not", text)
    text = re.sub(r"can\'t", "can not", text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    return text

In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
stop_words= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

In [ ]:
def preprocess_text(text):
    text = re.sub(r"http\S+", "", text)             # removing website links
    text = BeautifulSoup(text, 'lxml').get_text()   # removing html tags
    text = decontract(text)                         # decontracting
    text = re.sub("\S*\d\S*", "", text).strip()     # removing the words with numeric digits
    text = re.sub('[^A-Za-z]+', ' ', text)          # removing non-word characters
    text = text.lower()                             # converting to lower case
    text = [word for word in text.split(" ") if not word in stop_words] # removing stop words
    text = [lemmatizer.lemmatize(token, "v") for token in text] #Lemmatization
    text = " ".join(text)
    text.strip()
    return text

def convert_task(text):
    if text == "HOF":
      return 1
    return 0

In [ ]:
hasoc_data["text"] = hasoc_data["text"].apply(lambda x: preprocess_text(x))
hasoc_data["task_1"] = hasoc_data["task_1"].apply(lambda x: convert_task(x))
aakash_test["text"] = aakash_test["text"].apply(lambda x: preprocess_text(x))
aakash_test["task_1"] = aakash_test["task_1"].apply(lambda x: convert_task(x))
test_hasoc["text"] = test_hasoc["text"].apply(lambda x: preprocess_text(x))

In [ ]:
hasoc_data["text"].head()

0     wealth make not able start make money sustain...
1          technically still turn back clock dick head
2     vmbjp narendramodi jpnadda amitshah dilipghos...
3               krtoprak yigit soldier japan dick head
4     blueheartedly would better ask think sleazy s...
Name: text, dtype: object

In [ ]:
hasoc_data["task_1"].head()

0    1
1    1
2    0
3    1
4    1
Name: task_1, dtype: int64

In [ ]:
hasoc_data["task_1"].value_counts()

1    2501
0    1342
Name: task_1, dtype: int64

In [ ]:
aakash_test["task_1"].value_counts()

1    230
0     70
Name: task_1, dtype: int64

In [ ]:
hasoc_data.drop(["Unnamed: 0","_id","task_2"],inplace=True,axis=1)

In [ ]:
hasoc_data.head()

,text,task_1
0,wealth make not able start make money sustain...,1
1,technically still turn back clock dick head,1
2,vmbjp narendramodi jpnadda amitshah dilipghos...,0
3,krtoprak yigit soldier japan dick head,1
4,blueheartedly would better ask think sleazy s...,1


In [ ]:
train_df = hasoc_data
test_df = aakash_test
print("Training data size : ", train_df.shape)
print("Test data size : ", test_df.shape)

Training data size :  (3843, 2)
Test data size :  (300, 12)


#Paramters

In [ ]:
top_words = 10000
tokenizer = Tokenizer(num_words=top_words)
tokenizer.fit_on_texts(train_df['text'])
word_index = tokenizer.word_index
list_tokenized_train = tokenizer.texts_to_sequences(train_df['text'])

max_review_length = 130
X_train = pad_sequences(list_tokenized_train, maxlen=max_review_length)
y_train = train_df['task_1']
embedding_vecor_length = 32

In [ ]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(hasoc_data["text"][3]))
print(hasoc_data['text'][3])

? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ?
 krtoprak yigit soldier japan dick head


#Bidirectional LSTM 

In [ ]:
model3 = tf.keras.Sequential([
    tf.keras.layers.Embedding(top_words+1, embedding_vecor_length, input_length=max_review_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
checkpoint_path = "/tmp/callback/checkpoint-lstm-bi/lstm-bi-{epoch:04d}.ckpt"
batch_size = 64

In [ ]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_path,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=False,
    save_freq=batch_size)

In [ ]:
model3.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model3.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 130, 32)           320032    
_________________________________________________________________
bidirectional (Bidirectional (None, 130, 128)          49664     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                41216     
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 415,137
Trainable params: 415,137
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history3 = model3.fit(X_train,y_train, epochs=100, batch_size=batch_size,callbacks=[model_checkpoint_callback])


Epoch 1/100
61/61 [==============================] - 18s 114ms/step - loss: 0.6451 - accuracy: 0.6430
Epoch 2/100
61/61 [==============================] - 7s 116ms/step - loss: 0.4347 - accuracy: 0.8098
Epoch 3/100
61/61 [==============================] - 7s 116ms/step - loss: 0.2364 - accuracy: 0.9157
Epoch 4/100
61/61 [==============================] - 7s 115ms/step - loss: 0.1400 - accuracy: 0.9599
Epoch 5/100
61/61 [==============================] - 7s 116ms/step - loss: 0.0898 - accuracy: 0.9750
Epoch 6/100
61/61 [==============================] - 7s 116ms/step - loss: 0.0665 - accuracy: 0.9818
Epoch 7/100
61/61 [==============================] - 7s 116ms/step - loss: 0.0511 - accuracy: 0.9880
Epoch 8/100
61/61 [==============================] - 7s 116ms/step - loss: 0.0436 - accuracy: 0.9896
Epoch 9/100
61/61 [==============================] - 7s 117ms/step - loss: 0.0377 - accuracy: 0.9922
Epoch 10/100
61/61 [==============================] - 7s 117ms/step - loss: 0.0309 - accur

In [ ]:
e = model3.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)
vocab_size = weights.shape[0]

(10001, 32)


Result and plot of Bidirectional LSTM

In [ ]:
list_tokenized_test = tokenizer.texts_to_sequences(test_df['text'])
X_test = pad_sequences(list_tokenized_test, maxlen=max_review_length)
y_test = test_df['task_1']
prediction = model3.predict(X_test)
y_pred = (prediction > 0.5)
print("Accuracy of the model : ", accuracy_score(y_pred, y_test))
print('F1-score: ', f1_score(y_pred, y_test))
print('Confusion matrix:')
confusion_matrix(y_test,y_pred)

Accuracy of the model :  0.7566666666666667
F1-score:  0.8298368298368298
Confusion matrix:


array([[ 49,  21],
       [ 52, 178]])

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.49      0.70      0.57        70
           1       0.89      0.77      0.83       230

    accuracy                           0.76       300
   macro avg       0.69      0.74      0.70       300
weighted avg       0.80      0.76      0.77       300



Word Embeddings

LSTM Bidirectional Callback

In [ ]:
model3_final = tf.keras.Sequential([
    tf.keras.layers.Embedding(top_words+1, embedding_vecor_length, input_length=max_review_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model3_final.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model3_final.load_weights("/tmp/callback/checkpoint-lstm-bi/lstm-bi-0040.ckpt")

In [ ]:
prediction3_final = model3_final.predict(X_test)
y_pred_final = (prediction3_final > 0.5)
print("Accuracy of the model : ", accuracy_score(y_pred_final, y_test))
print('F1-score: ', f1_score(y_pred_final, y_test))
print('Confusion matrix:')
confusion_matrix(y_test,y_pred_final)

Accuracy of the model :  0.7833333333333333
F1-score:  0.8526077097505669
Confusion matrix:


array([[ 47,  23],
       [ 42, 188]])

In [ ]:
print(classification_report(y_test, y_pred_final))

              precision    recall  f1-score   support

           0       0.51      0.74      0.60        70
           1       0.91      0.78      0.84       230

    accuracy                           0.77       300
   macro avg       0.71      0.76      0.72       300
weighted avg       0.82      0.77      0.79       300



#Test submission

In [ ]:
list_tokenized_test = tokenizer.texts_to_sequences(test_hasoc["text"])
subm = pad_sequences(list_tokenized_test, maxlen=max_review_length)
y_submission = model3_final.predict(subm)
y_submission = (y_submission > 0.5)

In [ ]:
print(y_submission)
zeros = 0
ones = 0
submission = []
for i in range(len(y_submission)):
  if y_submission[i] == 0:
    zeros += 1
    submission.append("NOT")
    
  else:
    ones += 1
    submission.append("HOF")

print("zeros ",zeros)
print("ones ",ones)


[[False]
 [ True]
 [ True]
 ...
 [ True]
 [False]
 [ True]]
zeros  404
ones  877


Rohan SVM model had zeros = 314 and ones = 967

#Submission File

In [ ]:
submission = {'id': test_hasoc._id, 'label':submission}
submission = pd.DataFrame(submission)

In [ ]:
submission.head()

,id,label
0,60c5d6bf5659ea5e55deffcb,NOT
1,60c5d6bf5659ea5e55df028c,HOF
2,60c5d6bf5659ea5e55def377,HOF
3,60c5d6bf5659ea5e55def4c7,NOT
4,60c5d6bf5659ea5e55df01a6,NOT


In [ ]:
submission.describe()

,id,label
count,1281,1281
unique,1281,2
top,60c5d6bf5659ea5e55def6d7,HOF
freq,1,877


In [ ]:
submission.to_csv('drive/MyDrive/ICHCL_baseline/Submissions/submission_bidi_lstm_final.csv', index = False)